In [117]:
import pandas as pd
import numpy as np
import pickle
import os

import warnings
warnings.filterwarnings('ignore')

# Load

In [118]:
FOLDER = '../bin'
raw_bert_path = os.path.join(FOLDER, 'bert', 'experiments_raw')
cleaned_bert_path = os.path.join(FOLDER, 'bert', 'final')
raw_indobert_path = os.path.join(FOLDER, 'indobert', 'experiments_raw')
cleaned_indobert_path = os.path.join(FOLDER, 'indobert', 'final')

path = {
    'data_raw': {
        'bert-base': {
            'max-256-lr-1e-4': os.path.join(raw_bert_path, '2021-09-24 13.24.48', 'y_pred.pkl'),
            'max-512-lr-1e-4': os.path.join(raw_bert_path, '2021-09-24 14.15.34', 'y_pred.pkl'),
            'max-256-lr-3e-6': os.path.join(raw_bert_path, '2021-09-24 14.42.00', 'y_pred.pkl'),
            'max-512-lr-3e-6': os.path.join(raw_bert_path, '2021-09-24 15.32.53', 'y_pred.pkl')
        },
        'bert-base-multilingual': {
            'max-256-lr-1e-4': os.path.join(raw_bert_path, '2021-09-24 16.02.37', 'y_pred.pkl'),
            'max-512-lr-1e-4': os.path.join(raw_bert_path, '2021-09-24 16.55.21', 'y_pred.pkl'),
            'max-256-lr-3e-6': os.path.join(raw_bert_path, '2021-09-24 17.23.44', 'y_pred.pkl'),
            'max-512-lr-3e-6': os.path.join(raw_bert_path, '2021-09-24 18.16.40', 'y_pred.pkl') 
        },
        'indobert': {
            'max-256-lr-1e-4': os.path.join(raw_indobert_path, '2021-09-24 18.32.42', 'prediction.csv'),
            'max-512-lr-1e-4': os.path.join(raw_indobert_path, '2021-09-24 18.47.20', 'prediction.csv'),
            'max-256-lr-3e-6': os.path.join(raw_indobert_path, '2021-09-24 19.02.12', 'prediction.csv'),
            'max-512-lr-3e-6': os.path.join(raw_indobert_path, '2021-09-24 19.18.08', 'prediction.csv') 
        }
    },
    'data_cleaned': {
        'bert-base': {
            'max-512-lr-3e-6': os.path.join(FOLDER, 'bert', 'final', 'bert-base-uncased', 'y_pred.pkl')
        },
        'bert-base-multilingual': {
            'max-512-lr-3e-6': os.path.join(FOLDER, 'bert', 'final', 'bert-base-multilingual-uncased', 'y_pred.pkl')
        },
        'indobert': {
            'max-512-lr-3e-6': os.path.join(FOLDER, 'indobert', 'final', 'prediction.csv')
        }
    }
}

In [119]:
raw_df_test = pd.read_csv('../data/test.csv')
cleaned_df_test = pd.read_csv('../data/cleaned_test.csv')

In [120]:
df_test = pd.DataFrame({
    'raw_text': raw_df_test['text_a'],
    'cleaned_text': cleaned_df_test['text_a'],
    'label': cleaned_df_test['label']
})

df_without_label = df_test.drop(columns='label')

In [121]:
IDX2LABEL = {0: 'no', 1: 'yes'}
LABEL2IDX = {'no': 0, 'yes': 1}

# Prediction

In [122]:
def conv_pickle_to_csv(path):
    temp_df = df_without_label.copy()
    y_pred = pickle.load(open(path, 'rb')).flatten()
    y_pred = [IDX2LABEL[el] for el in y_pred]
    temp_df['prediction'] = y_pred
    return temp_df
    
def load_pred_csv(path):
    temp_df = df_without_label.copy()
    y_pred = pd.read_csv(path)['label']
    temp_df['prediction'] = y_pred
    return temp_df

In [123]:
pickle_out = set(['bert-base', 'bert-base-multilingual'])

raw_prediction = {}

for model_name, params in path['data_raw'].items():
    raw_prediction[model_name] = {}
    for params_name, path_prediction in params.items():
        if model_name in pickle_out:
            raw_prediction[model_name][params_name] = conv_pickle_to_csv(path_prediction)
        else:
            raw_prediction[model_name][params_name] = load_pred_csv(path_prediction)
            
cleaned_prediction = {}
for model_name, params in path['data_cleaned'].items():
    cleaned_prediction[model_name] = {}
    for params_name, path_prediction in params.items():
        if model_name in pickle_out:
            cleaned_prediction[model_name][params_name] = conv_pickle_to_csv(path_prediction)
        else:
            cleaned_prediction[model_name][params_name] = load_pred_csv(path_prediction)


# Scores

In [124]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score

evaluations = {
    'accuracy': accuracy_score,
    'precision': precision_score,
    'recall': recall_score,
    'classification_report': classification_report,
    'confusion_matrix': confusion_matrix
}

raw_scores = {}
y_true = df_test['label']

for model_name, params in raw_prediction.items():
    raw_scores[model_name] = {}
    for param_name, prediction in params.items():
        raw_scores[model_name][param_name] = {}
        y_pred = raw_prediction[model_name][param_name]['prediction']
        for evaluation_name, func in evaluations.items():
            if evaluation_name in ['precision', 'recall']:
                raw_scores[model_name][param_name][evaluation_name] = func(y_true, y_pred, pos_label='yes')
            else:
                raw_scores[model_name][param_name][evaluation_name] = func(y_true, y_pred)
                
cleaned_scores = {}

for model_name, params in cleaned_prediction.items():
    cleaned_scores[model_name] = {}
    for param_name, prediction in params.items():
        cleaned_scores[model_name][param_name] = {}
        y_pred = cleaned_prediction[model_name][param_name]['prediction']
        for evaluation_name, func in evaluations.items():
            if evaluation_name in ['precision', 'recall']:
                cleaned_scores[model_name][param_name][evaluation_name] = func(y_true, y_pred, pos_label='yes')
            else:
                cleaned_scores[model_name][param_name][evaluation_name] = func(y_true, y_pred)

In [125]:
def summarize_score(scores, printed_evaluation_keys=['accuracy', 'precision', 'recall']):
    for model_name, params in scores.items():
        print(model_name)
        for param_name, evaluations in params.items():
            print('\t', param_name)
            for evaluation_name in printed_evaluation_keys:
                if evaluation_name in ['confusion_matrix', 'classification_report']:
                    print('\t\t', evaluation_name)
                    print(evaluations[evaluation_name])
                else:
                    print('\t\t', evaluation_name, ': ', evaluations[evaluation_name])

In [126]:
summarize_score(raw_scores)

bert-base
	 max-256-lr-1e-4
		 accuracy :  0.7475
		 precision :  0.0
		 recall :  0.0
	 max-512-lr-1e-4
		 accuracy :  0.7475
		 precision :  0.0
		 recall :  0.0
	 max-256-lr-3e-6
		 accuracy :  0.8492857142857143
		 precision :  0.6853055916775033
		 recall :  0.7454031117397454
	 max-512-lr-3e-6
		 accuracy :  0.8435714285714285
		 precision :  0.6662546353522868
		 recall :  0.7623762376237624
bert-base-multilingual
	 max-256-lr-1e-4
		 accuracy :  0.7475
		 precision :  0.0
		 recall :  0.0
	 max-512-lr-1e-4
		 accuracy :  0.7475
		 precision :  0.0
		 recall :  0.0
	 max-256-lr-3e-6
		 accuracy :  0.8453571428571428
		 precision :  0.6658595641646489
		 recall :  0.7779349363507779
	 max-512-lr-3e-6
		 accuracy :  0.8457142857142858
		 precision :  0.6546681664791901
		 recall :  0.8231966053748232
indobert
	 max-256-lr-1e-4
		 accuracy :  0.7475
		 precision :  0.0
		 recall :  0.0
	 max-512-lr-1e-4
		 accuracy :  0.7475
		 precision :  0.0
		 recall :  0.0
	 max-256-lr-3e-6
		

In [128]:
summarize_score(cleaned_scores)

bert-base
	 max-512-lr-3e-6
		 accuracy :  0.8589285714285714
		 precision :  0.6893203883495146
		 recall :  0.8033946251768034
bert-base-multilingual
	 max-512-lr-3e-6
		 accuracy :  0.8492857142857143
		 precision :  0.6574585635359116
		 recall :  0.8415841584158416
indobert
	 max-512-lr-3e-6
		 accuracy :  0.8875
		 precision :  0.7525773195876289
		 recall :  0.826025459688826


In [129]:
summarize_score(cleaned_scores, printed_evaluation_keys=['confusion_matrix'])

bert-base
	 max-512-lr-3e-6
		 confusion_matrix
[[1837  256]
 [ 139  568]]
bert-base-multilingual
	 max-512-lr-3e-6
		 confusion_matrix
[[1783  310]
 [ 112  595]]
indobert
	 max-512-lr-3e-6
		 confusion_matrix
[[1901  192]
 [ 123  584]]


In [131]:
summarize_score(raw_scores, printed_evaluation_keys=['confusion_matrix'])

bert-base
	 max-256-lr-1e-4
		 confusion_matrix
[[2093    0]
 [ 707    0]]
	 max-512-lr-1e-4
		 confusion_matrix
[[2093    0]
 [ 707    0]]
	 max-256-lr-3e-6
		 confusion_matrix
[[1851  242]
 [ 180  527]]
	 max-512-lr-3e-6
		 confusion_matrix
[[1823  270]
 [ 168  539]]
bert-base-multilingual
	 max-256-lr-1e-4
		 confusion_matrix
[[2093    0]
 [ 707    0]]
	 max-512-lr-1e-4
		 confusion_matrix
[[2093    0]
 [ 707    0]]
	 max-256-lr-3e-6
		 confusion_matrix
[[1817  276]
 [ 157  550]]
	 max-512-lr-3e-6
		 confusion_matrix
[[1786  307]
 [ 125  582]]
indobert
	 max-256-lr-1e-4
		 confusion_matrix
[[2093    0]
 [ 707    0]]
	 max-512-lr-1e-4
		 confusion_matrix
[[2093    0]
 [ 707    0]]
	 max-256-lr-3e-6
		 confusion_matrix
[[1901  192]
 [ 140  567]]
	 max-512-lr-3e-6
		 confusion_matrix
[[1883  210]
 [ 125  582]]


# Deep Evaluation

In [138]:
cleaned_prediction.keys()

dict_keys(['bert-base', 'bert-base-multilingual', 'indobert'])

In [139]:
df_evaluation = df_test.copy()
df_evaluation['bert-base'] = cleaned_prediction['bert-base']['max-512-lr-3e-6']['prediction']
df_evaluation['bert-base-multilingual'] = cleaned_prediction['bert-base-multilingual']['max-512-lr-3e-6']['prediction']
df_evaluation['indobert'] = cleaned_prediction['indobert']['max-512-lr-3e-6']['prediction']

In [144]:
def wrong_preds(model_name):
    return df_evaluation[df_evaluation['label'] != df_evaluation[model_name]][['raw_text', 'cleaned_text', 'label', model_name]]

In [145]:
wrongs = {
    'bert-base': wrong_preds('bert-base'),
    'bert-base-multilingual': wrong_preds('bert-base-multilingual'),
    'indobert': wrong_preds('indobert')
}

In [149]:
def print_text_preds(df, model_name, text_col='cleaned_text'):
    for text, preds in zip(df[text_col], df[model_name]):
        print(f'{text} -- {preds}')

In [150]:
print_text_preds(wrongs['bert-base'], 'bert-base')

buka data hp disalahin gak membantu nyalah2in kondisi kesehatan pasien positif corona depok dirawat rspi sulianti saroso membaik wali kota depok menyebut nya mengalami stres -- no
lgi marak virus corona diindo nya depok direktur kantor org depok waswas gitu pagi disediain jamu jahe dibeliin langsung nya disediain ditiap meja dipastiin smua karyawan nya minum jamu -- yes
udah waspada sengaja ambil krl keberangkatan depok sepi menjauhi orang gak pake masker dalem krl sampe kantor tetep didiskriminasi diblg gue kota terjangkit -- yes
mantap corona depok permintaan sereh langsung gacoooor pedagang pasar nyiapin stok -- no
tetangga 2 pasien corona depok diimbau rumah 14 14 warga tinggal rumah pasien corona depok jawa barat diimbau aktivitas petugas dinas kesehatan sumber merdeka com -- yes
corona depok puncak gunung es -- no
putraerlangga gubernur jabar ridwankamil virus corona bln hadir kota depok panik beliau warga antisipasi aja -- yes
ngapaen jg y w ngetweet jam 23 50 bahas corona conce

In [151]:
print_text_preds(wrongs['bert-base-multilingual'], 'bert-base-multilingual')

kenapaa mendengar kabar salah wni positif corona depok dimana tinggal ku ku kawatir takut -- no
udah waspada sengaja ambil krl keberangkatan depok sepi menjauhi orang gak pake masker dalem krl sampe kantor tetep didiskriminasi diblg gue kota terjangkit -- yes
hotradero detonator sayang nya rakyat byk yg meragukan kualitas bandara utk menangkal penyebaran virus gerbang negara btw org jepang yg mejangkiti warga depok lolos ya bang gejala kelalaian -- yes
micheladamnp ya allah yg 28 mbak kemarin koar2 jelek2kan jepun umumkan 2 korban depok pasang badan tuan suspect corona rsud cianjur kerja rumah makan cina jakarta dikunjungi wna rezimtutupicorona -- yes
tetangga 2 pasien corona depok diimbau rumah 14 14 warga tinggal rumah pasien corona depok jawa barat diimbau aktivitas petugas dinas kesehatan sumber merdeka com -- yes
corona depok puncak gunung es -- no
putraerlangga gubernur jabar ridwankamil virus corona bln hadir kota depok panik beliau warga antisipasi aja -- yes
2020 ternyta indon

In [152]:
print_text_preds(wrongs['indobert'], 'indobert')

buka data hp disalahin gak membantu nyalah2in kondisi kesehatan pasien positif corona depok dirawat rspi sulianti saroso membaik wali kota depok menyebut nya mengalami stres -- no
depok dikarantina kataidi -- yes
hotradero detonator sayang nya rakyat byk yg meragukan kualitas bandara utk menangkal penyebaran virus gerbang negara btw org jepang yg mejangkiti warga depok lolos ya bang gejala kelalaian -- yes
micheladamnp ya allah yg 28 mbak kemarin koar2 jelek2kan jepun umumkan 2 korban depok pasang badan tuan suspect corona rsud cianjur kerja rumah makan cina jakarta dikunjungi wna rezimtutupicorona -- yes
mantap corona depok permintaan sereh langsung gacoooor pedagang pasar nyiapin stok -- no
tetangga 2 pasien corona depok diimbau rumah 14 14 warga tinggal rumah pasien corona depok jawa barat diimbau aktivitas petugas dinas kesehatan sumber merdeka com -- yes
corona depok puncak gunung es -- no
putraerlangga gubernur jabar ridwankamil virus corona bln hadir kota depok panik beliau warg